In [ ]:
%%!pip install transformers

from transformers import AutoModelForCausalLM, AutoTokenizer


device = "cuda" # the device to load the model onto

def load_model():
    """
    Loads the tokenizer and model from the specified pretrained model.

    This function initializes and returns the tokenizer and model 
    using the 'mistralai/Mistral-7B-Instruct-v0.2' pretrained model.

    Returns:
        tuple: A tuple containing the tokenizer and model instances.
    """
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
    return tokenizer, model


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Absolutely! Here's a simple and classic homemade mayonnaise recipe:

Ingredients:
- 1 egg yolk (at room temperature)
- 1 tablespoon of dijon mustard
- 1 cup of vegetable oil (preferably safflower or canola oil for its neutral flavour)
- 1-2 tablespoons of white wine vinegar or fresh lemon juice
- Salt to taste

Instructions:
1. In a clean bowl, whisk together the egg yolk and mustard until they are well combined.
2. Start adding the oil very slowly, drop by drop, while continuously whisking the mixture. This is called tempering the egg and it helps to prevent the oil from separating from the yolk.
3. Once the oil is fully incorporated, you can start adding it in a thin stream, still whisking constantly.
4. When all the oil has been a

In [ ]:
# Accessing the first layer
model._modules
model.modules
for name, module in model.named_modules():
    print(name, module)

OrderedDict([('model',
              MistralModel(
                (embed_tokens): Embedding(32000, 4096)
                (layers): ModuleList(
                  (0-31): 32 x MistralDecoderLayer(
                    (self_attn): MistralSdpaAttention(
                      (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
                      (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
                      (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
                      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
                      (rotary_emb): MistralRotaryEmbedding()
                    )
                    (mlp): MistralMLP(
                      (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
                      (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
                      (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
 

In [54]:
activations = {}

def save_activation(layer_name):
    def hook(model, input, output):
        # Check if the output is a tuple of tensors
        print("output", output)

        # Single tensor output, detach it and move to GPU
        activations[layer_name] = output.detach().to('cuda')
    return hook



# Assuming `model` is an instance of `MistralForCausalLM`
# and has an attribute `model` which is an instance of `MistralModel`
for i, layer in enumerate(model.model.layers):
    # Register the hook to each decoder layer
    layer.register_forward_hook(save_activation(f'decoder_layer_{i}'))


In [49]:
# List of inputs 

input_texts = [ {"role": "user", "content": "John is an incredibly complex individual, with a very rich personality."}] 

In [51]:
# Tokenize your input texts (you need to have a list of texts prepared)
tokenized_input = tokenizer.apply_chat_template(input_texts, return_tensors="pt")


In [56]:
all_activations = []

for i, tokenized_input in enumerate(tokenized_inputs):
    # Forward pass through the model
    tokenized_input = tokenized_input.to(device)
    print(tokenized_input)
    generated_ids = model.generate(tokenized_input, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded[0])

    # Store the activations for this input
    all_activations.append(activations.copy())

    # Clear the activations for the next input
    activations.clear()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1,  2215,   349,   264,  1215, 10032,   676, 28723,   650,  3208,
           298,  2590,   347,  4592,   438,   771, 28725,   562,  3573,  2940,
           298,   776,  1250,   264,  4229,   533,   656]], device='cuda:0')
output (tensor([[[-0.0004,  0.0013, -0.1514,  ...,  0.0002,  0.0024,  0.0057],
         [ 0.0011,  0.0063,  0.0039,  ...,  0.0043, -0.0082,  0.0073],
         [ 0.0040, -0.0006,  0.0024,  ...,  0.0042, -0.0020, -0.0031],
         ...,
         [-0.0020,  0.0042,  0.0037,  ..., -0.0076, -0.0051, -0.0047],
         [ 0.0029,  0.0035,  0.0012,  ...,  0.0061, -0.0049,  0.0035],
         [-0.0011,  0.0024, -0.0067,  ...,  0.0005,  0.0018,  0.0045]]],
       device='cuda:0'), DynamicCache())


AttributeError: 'tuple' object has no attribute 'detach'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # Replace `model` with your actual model variable

# Define hook with device management
def save_activation(layer_name):
    def hook(model, input, output):
        activations[layer_name] = output.detach().to(device)  # Ensures activations are on the same device
    return hook

# Register hook
for layer_name, layer in model.named_modules():
    layer.register_forward_hook(save_activation(layer_name))

# Prepare inputs and run the model
for i, tokenized_input in enumerate(tokenized_inputs):
    tokenized_input = tokenized_input.to(device)  # Ensure input is on the same device
    output = model(tokenized_input)

    # ... Save or process your activations


In [ ]:
# Getting activations from the model 

def get_acts(statements, tokenizer, model, layers, device):
    """
    Get given layer activations for the statements. 
    Return dictionary of stacked activations.
    """
    # attach hooks
    hooks, handles = [], []
    for layer in layers:
        hook = Hook()
        handle = model.model.layers[layer].register_forward_hook(hook)
        hooks.append(hook), handles.append(handle)
    
    # get activations
    acts = {layer : [] for layer in layers}
    for statement in tqdm(statements):
        input_ids = tokenizer.encode(statement, return_tensors="pt").to(device)
        model(input_ids)
        for layer, hook in zip(layers, hooks):
            acts[layer].append(hook.out[0, -1])
    
    for layer, act in acts.items():
        acts[layer] = t.stack(act).float()
    
    # remove hooks
    for handle in handles:
        handle.remove()
    
    return acts


In [ ]:
# Activation script 
import torch as t
from transformers import LlamaForCausalLM, LlamaTokenizer
import argparse
import pandas as pd
from tqdm import tqdm
import os
import configparser


if __name__ == "__main__":
    """
    read statements from dataset, record activations in given layers, and save to specified files
    """
    parser = argparse.ArgumentParser(description="Generate activations for statements in a dataset")
    parser.add_argument("--model", default="13B",
                        help="Size of the model to use. Options are 7B or 30B")
    parser.add_argument("--layers", nargs='+', 
                        help="Layers to save embeddings from")
    parser.add_argument("--datasets", nargs='+',
                        help="Names of datasets, without .csv extension")
    parser.add_argument("--output_dir", default="acts",
                        help="Directory to save activations to")
    parser.add_argument("--device", default="cpu")
    args = parser.parse_args()

    t.set_grad_enabled(False)
    
    tokenizer, model = load_model(args.model, args.device)
    for dataset in args.datasets:
        statements = load_data(dataset)
        layers = [int(layer) for layer in args.layers]
        if layers == [-1]:
            layers = list(range(len(model.model.layers)))
        save_dir = f"{args.output_dir}/{args.model}/{dataset}/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        for idx in range(0, len(statements), 25):
            acts = get_acts(statements[idx:idx + 25], tokenizer, model, layers, args.device)
            for layer, act in acts.items():
                    t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")

In [9]:
import pandas as pd
!pip3 install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [10]:
df = pd.read_excel('/Users/matthieu/Code/personal_ai/X Retail & Commercial Report 01102023 - 31122023.xlsx')


In [11]:
df

,Customer reference,Customer type,Service centre code,Service centre name,Brand,Type,Vehicle registration,Article short name,Article name,Document number,Document Insert date,Qty sold,Commercial discount,Discount 2 (%),Gross Sales (Local curr.),Net Sales (Local curr.),Centre Margin (Local curr.)
0,NaN,RETAIL,A366,Tamworth Retail,EUROMASTER,T016 - Car & Van,BT69BTO,SECWBA,WHEEL BALANCE-CAR(A),A3660577321,2023-11-03 17:27:52.000,3.0,0.0,0.0000,23.75,23.75,23.75
1,NaN,RETAIL,A050,Andover,EUROMASTER,T016 - Car & Van,WR57 PTU,SECSD,SCRAP DISPOSAL CAR VAN 4X4,A0500530728,2023-11-16 12:58:58.999,1.0,0.0,0.0000,2.05,2.05,2.05
2,NaN,RETAIL,A625,Kilmarnock,EUROMASTER,T027 - Batteries,EC52CON,BATT096AGMNAP,BATTERY 096AGM 3YR NAPA,A6250394707,2023-11-29 15:26:12.999,1.0,0.0,0.1000,170.83,153.75,42.59
3,NaN,RETAIL,A337,Halesowen,EUROMASTER,T047 - MOT Testing,VE69FEF,SEAMOT,MOT TEST - CLASS 4,A3370265999,2023-11-21 11:58:30.999,1.0,0.0,0.0000,32.00,32.00,32.00
4,NaN,RETAIL,A462,Nottingham Castle Boulevard,EUROMASTER,T047 - MOT Testing,FG66BHA,SEAMOT,MOT TEST - CLASS 4,A4620458467,2023-10-16 12:12:27.000,1.0,0.0,0.2707,54.85,40.00,40.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873802,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,LB20KLC,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800232342,2023-12-21 20:17:47.000,1.0,0.0,0.0000,60.00,60.00,60.00
873803,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,AY67OXN,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800223258,2023-10-25 11:15:23.999,1.0,0.0,0.0000,58.50,58.50,58.50
873804,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,LF19BHV,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800222714,2023-10-21 17:29:56.000,1.0,0.0,0.0000,58.50,58.50,58.50
873805,A00954454,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,WX71KLO,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800222710,2023-10-21 13:43:55.000,1.0,0.0,0.0000,54.06,54.06,54.06


In [12]:
df

,Customer reference,Customer type,Service centre code,Service centre name,Brand,Type,Vehicle registration,Article short name,Article name,Document number,Document Insert date,Qty sold,Commercial discount,Discount 2 (%),Gross Sales (Local curr.),Net Sales (Local curr.),Centre Margin (Local curr.)
0,NaN,RETAIL,A366,Tamworth Retail,EUROMASTER,T016 - Car & Van,BT69BTO,SECWBA,WHEEL BALANCE-CAR(A),A3660577321,2023-11-03 17:27:52.000,3.0,0.0,0.0000,23.75,23.75,23.75
1,NaN,RETAIL,A050,Andover,EUROMASTER,T016 - Car & Van,WR57 PTU,SECSD,SCRAP DISPOSAL CAR VAN 4X4,A0500530728,2023-11-16 12:58:58.999,1.0,0.0,0.0000,2.05,2.05,2.05
2,NaN,RETAIL,A625,Kilmarnock,EUROMASTER,T027 - Batteries,EC52CON,BATT096AGMNAP,BATTERY 096AGM 3YR NAPA,A6250394707,2023-11-29 15:26:12.999,1.0,0.0,0.1000,170.83,153.75,42.59
3,NaN,RETAIL,A337,Halesowen,EUROMASTER,T047 - MOT Testing,VE69FEF,SEAMOT,MOT TEST - CLASS 4,A3370265999,2023-11-21 11:58:30.999,1.0,0.0,0.0000,32.00,32.00,32.00
4,NaN,RETAIL,A462,Nottingham Castle Boulevard,EUROMASTER,T047 - MOT Testing,FG66BHA,SEAMOT,MOT TEST - CLASS 4,A4620458467,2023-10-16 12:12:27.000,1.0,0.0,0.2707,54.85,40.00,40.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873802,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,LB20KLC,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800232342,2023-12-21 20:17:47.000,1.0,0.0,0.0000,60.00,60.00,60.00
873803,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,AY67OXN,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800223258,2023-10-25 11:15:23.999,1.0,0.0,0.0000,58.50,58.50,58.50
873804,A00935660,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,LF19BHV,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800222714,2023-10-21 17:29:56.000,1.0,0.0,0.0000,58.50,58.50,58.50
873805,A00954454,COMMERCIAL,A880,S1 - South Wales & Central,EUROMASTER,T016 - Car & Van,WX71KLO,SEVRETORQB,RE-TORQUE STAND TIME ONWH VAN,A8800222710,2023-10-21 13:43:55.000,1.0,0.0,0.0000,54.06,54.06,54.06
